# Laboratorio: Modelos del lenguaje con RNNs (Caracteres)

[Enlace al cuaderno en el Github](https://github.com/aaronmed/7ro/blob/master/TAREA%208%20(23-24)%20RNN%20USANDO%20EMBEDDINGS%20(TRAFALGAR)/7RO_RNN_Caracteres_AaronMedinaMelian.ipynb)

Aarón Medina Melián

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Portada_Trafalgar_%281873%29.jpg/800px-Portada_Trafalgar_%281873%29.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano de Trafalgar, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro Trafalgar, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [3]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import io

In [4]:
## TU CÓDIGO AQUÍ
path = keras.utils.get_file(fname="trafalgar.txt", origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ")

Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [5]:
## TU CÓDIGO AQUÍ
text = open(path, encoding="utf8").read().lower()

Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado

In [6]:
## TU CÓDIGO AQUÍ
print("Longitud del texto: {}".format(len(text)))
print(text[0:300])

Longitud del texto: 300039
-i-

se me permitirá que antes de referir el gran suceso de que fui testigo,
diga algunas palabras sobre mi infancia, explicando por qué extraña
manera me llevaron los azares de la vida a presenciar la terrible
catástrofe de nuestra marina.

al hablar de mi nacimiento, no imitaré a la mayor parte de


In [7]:
text = text[5:len(text)]

In [8]:
fin = "fin de trafalgar"
posicion = text.find(fin)


print(len(fin))
print(posicion)

text[posicion:len(text)]

16
299920


'fin de trafalgar\n\nmadrid, enero-febrero 1873.\n\nbenito pérez galdós; edición ilustrada por enrique y arturo mélida\n'

In [9]:
text = text[0:posicion+len(fin)]

## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*Se me permitirá que antes de referir el gr*" -> predicción: **a**
* "*e me permitirá que antes de referir el gra*" -> predicción: **n**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: e me permitirá que antes de referir el gra

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: a

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante (PERO USANDO PALABRAS NO CARACTERES).

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de PALABRAS con la siguiente PALABRA a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* PALABRAS (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de las palabras y mapas de palabras

Antes que nada, necesitamos saber qué palabras aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_words* - 1 en el modelo. Obtener:


1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_words* - 1. Por ejemplo, {'se': 0, 'me': 1, ...}
3.   Diccionario reverso de índices a palabras: {0: 'se', 1: 'me', ...}


In [8]:
import re
from unidecode import unidecode

text = re.sub(r'[^A-Za-záéíóúüÁÉÍÓÚÜ\s]', '', text)
# Sustituir letras con tilde por su versión sin tilde
text = unidecode(text)

In [9]:
chars=sorted(list(set(text)))
char_indices=dict((c,i) for i,c in enumerate(chars))
indices_char=dict((i,c) for i,c in enumerate(chars))
print('Caracteres únicos: {}'.format(len(chars)))
print(char_indices)
print(indices_char)

Caracteres únicos: 28
{'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
{0: '\n', 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'w', 25: 'x', 26: 'y', 27: 'z'}


#### 2.2. Obtención de secuencias de entrada y palabra a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y las correspondientes palabras a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH palabras y la siguiente palabra a predecir. Una vez hecho, desplazarse una palabra a la derecha y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y las palabras a predecir en una variable ***next_words***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 2, tendríamos

* *sequences* = ["Don Quijote", "Quijote de"]
* *next_chars* = ['de', 'La']

In [10]:
# Definimos el tamaño de las secuencias. Puedes dejar este valor por defecto.
SEQ_LENGTH = 30
step = 1
sequences = []
next_chars = []

for i in range(0,len(text)-SEQ_LENGTH, step):
  sequences.append(text[i:i+SEQ_LENGTH])
  next_chars.append(text[i+SEQ_LENGTH])

In [11]:
print('Ejemplo: secuencia número 5:')
print(sequences[4])
print('Siguiente caracter:')
print(next_chars[4])

Ejemplo: secuencia número 5:
e permitira que antes de refer
Siguiente caracter:
i


Indicar el tamaño del training set que acabamos de generar.

In [12]:
## TU CÓDIGO AQUÍ
print('número de datos de training: {}'.format(len(sequences)))

número de datos de training: 288600


Como el libro es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [13]:
MAX_SEQUENCES = 500000

perm = np.random.permutation(len(sequences))
sequences, next_chars = np.array(sequences), np.array(next_chars)
sequences, next_chars = sequences[perm], next_chars[perm]
sequences, next_chars = list(sequences[:MAX_SEQUENCES]), list(next_chars[:MAX_SEQUENCES])

print(len(sequences))

288600


#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar *one-hot encoding* para nuestras palabras. Por ejemplo, si sólo tuviéramos 4 palabras (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_words)* e **y** tendrá shape *(num_sequences, num_words)*.



In [14]:
## TU CÓDIGO AQUÍ
NUM_CHARS = len(chars)  # Tu número de caracteres distintos aquí
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, NUM_CHARS))
y = np.zeros((NUM_SEQUENCES, NUM_CHARS))

## TU CÓDIGO AQUÍ
for i,sequence in enumerate(sequences): # i es el número de secuencia y sequence contiene los caracteres
  for t,char in enumerate(sequence): # t recorre la longitud de la secuencia y char cada caracter de la secuencia
    X[i,t,char_indices[char]]=1
  y[i,char_indices[next_chars[i]]]=1

In [15]:
print(sequences[0])
for j in range(SEQ_LENGTH):
  for k in range(NUM_CHARS):
    if (X[0,j,k]==1):
      print(indices_char[k],end='')

cisca pero cariosa y fraternal
cisca pero cariosa y fraternal

## 3. Definición del modelo y entrenamiento

Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [16]:
## TU CÓDIGO AQUÍ
vocab_size = len(chars)

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, vocab_size)))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               80384     
                                                                 
 dense (Dense)               (None, 28)                3612      
                                                                 
Total params: 83996 (328.11 KB)
Trainable params: 83996 (328.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [18]:
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(X, y, validation_split=0.5, batch_size=128, epochs=30, shuffle=True).history

Epoch 1/30
1128/1128 [==============================] - 51s 44ms/step - loss: 2.3308 - accuracy: 0.2882 - val_loss: 2.1043 - val_accuracy: 0.3368
Epoch 2/30
1128/1128 [==============================] - 54s 48ms/step - loss: 2.0125 - accuracy: 0.3655 - val_loss: 1.9649 - val_accuracy: 0.3775
Epoch 3/30
1128/1128 [==============================] - 52s 46ms/step - loss: 1.9112 - accuracy: 0.3965 - val_loss: 1.8876 - val_accuracy: 0.4012
Epoch 4/30
1128/1128 [==============================] - 53s 47ms/step - loss: 1.8390 - accuracy: 0.4196 - val_loss: 1.8420 - val_accuracy: 0.4178
Epoch 5/30
1128/1128 [==============================] - 53s 47ms/step - loss: 1.7791 - accuracy: 0.4396 - val_loss: 1.7882 - val_accuracy: 0.4402
Epoch 6/30
1128/1128 [==============================] - 58s 51ms/step - loss: 1.7245 - accuracy: 0.4579 - val_loss: 1.7395 - val_accuracy: 0.4566
Epoch 7/30
1128/1128 [==============================] - 54s 48ms/step - loss: 1.6755 - accuracy: 0.4738 - val_loss: 1.7077 -

In [19]:
import joblib # Para salvar el modelo
joblib.dump(model,'modelo_entrenado.pkl')
model2=joblib.load('modelo_entrenado.pkl')

In [20]:
results=model2.evaluate(X,y)
print(results)

9019/9019 [==============================] - 38s 4ms/step - loss: 1.3989 - accuracy: 0.5735
[1.3989105224609375, 0.5735412240028381]


Para ver cómo evoluciona nuestro modelo del lenguaje, vamos a generar texto según va entrenando. Para ello, vamos a programar una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo podemos ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtenemos un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que nuestros resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [21]:
## TU CÓDIGO AQUÍ
def sample(probs, temperature=1.0):
    probs = np.asarray(probs).astype('float64')
    
    probs = np.log(probs) / temperature
    
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)

Utilizando la función anterior y el modelo entrenado, vamos a añadir un callback a nuestro modelo para que, según vaya entrenando, veamos los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tenéis disponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.

Completa la función *generate_text* de modo que utilicemos el modelo y la función sample para generar texto.

NOTA: Cuando hagas model.predict, es aconsejable usar verbose=0 como argumento para evitar que la función imprima valores de salida.

In [22]:
#CÓDIGO Ejemplo: Predice el siguiente caracter de una secuencia aleatoria
# Primero, seleccionamos una secuencia al azar para empezar a predecir
# a partir de ella
start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
seed_text = text[start_pos:start_pos + SEQ_LENGTH]
print("Seed: {}".format(seed_text))
print()
#----------------------------------
# Aquí representomos la secuencia seed_text como codificación one-hot
X_pred = np.zeros((1, SEQ_LENGTH, NUM_CHARS))
# Construimos X_pred a partir de seed_text
for t,char in enumerate(seed_text): # t recorre la longitud de la secuencia y char cada caracter de la secuencia
  X_pred[0,t,char_indices[char]]=1
#-------------------------------------
#PREDICCIÓN
prediccion=model.predict(X_pred, batch_size=32, verbose=0)
print(indices_char[np.argmax(prediccion)],end='') # Muestra el caracter más probable para la sequencia 100
##################################################333

Seed: ndalucia en tal estado
que tod

o

In [23]:
print(seed_text)
seed_text=seed_text[1:SEQ_LENGTH-1]
print(seed_text)
seed_text+=indices_char[np.argmax(prediccion)]
print(seed_text)

ndalucia en tal estado
que tod
dalucia en tal estado
que to
dalucia en tal estado
que too


In [24]:
def genera_texto(texto, model, length):
  X_pred = np.zeros((1, SEQ_LENGTH, NUM_CHARS)) # Declaramos X_pred
  for k in range(length): # Genera el número de caracteres fijados en el parámetro length
    #Inicializamos X_pred a cero
    for a in range(SEQ_LENGTH):
      for b in range(NUM_CHARS):
         X_pred[0,a,b]=0
    # Aquí representomos la secuencia seed_text como codificación one-hot
    # Construimos X_pred a partir de seed_text
    for t,char in enumerate(texto): # t recorre la longitud de la secuencia y char cada caracter de la secuencia
      X_pred[0,t,char_indices[char]]=1
    #-------------------------------------
    #PREDICCIÓN
    prediccion=model.predict(X_pred, batch_size=32, verbose=0)
    print(indices_char[np.argmax(prediccion)],end='') # Muestra el caracter más probable para la sequencia dada
    #Actualizamos la cadena de entrada
    texto=texto[1:SEQ_LENGTH]
    texto+=indices_char[np.argmax(prediccion)]
    #texto+=indices_char[sample(prediccion[0],1)]
    #print(texto)

In [25]:
# Primero, seleccionamos una secuencia al azar para empezar a predecir
# a partir de ella
start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
seed_text = text[start_pos:start_pos + SEQ_LENGTH]
print('Seed:',seed_text)
print('Texto generado:')
genera_texto(seed_text, model2, 120)

Seed: erto estrechaba con su helada 
Texto generado:
de la escuadra de los marineros se almirante de los mas combatientes la escuadra completi con el combate de su andaner c

In [26]:
## TU CÓDIGO AQUÍ
TEMPERATURES_TO_TRY = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
GENERATED_TEXT_LENGTH = 300

In [27]:
def generate_text(seed_text, model, length, temperatura=1):
    generated = seed_text
    ### TU CÓDIGO AQUÍ
    ######################################################################
    texto=seed_text
    X_pred = np.zeros((1, SEQ_LENGTH, NUM_CHARS)) # Declaramos X_pred
    for k in range(length): # Genera el número de caracteres fijados en el parámetro length
      #Inicializamos X_pred a cero
      for a in range(SEQ_LENGTH):
        for b in range(NUM_CHARS):
          X_pred[0,a,b]=0
      # Aquí representomos la secuencia seed_text como codificación one-hot
      # Construimos X_pred a partir de seed_text
      for t,char in enumerate(texto): # t recorre la longitud de la secuencia y char cada caracter de la secuencia
        X_pred[0,t,char_indices[char]]=1
      #-------------------------------------
      #PREDICCIÓN
      prediccion=model.predict(X_pred, batch_size=32, verbose=0)
      # print(indices_char[np.argmax(prediccion)],end='') # Muestra el caracter más probable para la sequencia dada
      #Actualizamos la cadena de entrada
      texto=texto[1:SEQ_LENGTH]
      #texto+=indices_char[np.argmax(prediccion)]
      texto+=indices_char[sample(prediccion[0],temperatura)]
      # generated+=indices_char[np.argmax(prediccion)]
      generated+=indices_char[sample(prediccion[0],temperatura)]
    ######################################################################
    ### FIN DE TU CÓDIGO
    return generated

In [28]:
# Primero, seleccionamos una secuencia al azar para empezar a predecir
# a partir de ella
start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
seed_text = text[start_pos:start_pos + SEQ_LENGTH]
print('Seed:',seed_text)
print('Texto generado:')
generate_text(seed_text, model2, GENERATED_TEXT_LENGTH,temperatura=0.1)

Seed:  trabuco segun mis ideas con e
Texto generado:


' trabuco segun mis ideas con es mar de so que tabia de los meores de los nartes de la cardera yorque se aconta de la escuadra completinte que en el plcazar de su cabeza y da conversacion ye la cabpaada confestabon en la costaa en la cabdera d el combate de no se valvio a la escuadra de los marineros se almirante de luestra conte'

In [29]:
def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model2,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))
    print()


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Entrena ahora tu modelo. No te olvides de añadir *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


In [30]:
## TU CÓDIGO AQUÍ
history=model2.fit(X,y, validation_split=0.05, batch_size=128, epochs=10,shuffle=True,verbose=0,callbacks=[generation_callback]).history





------> Epoch: 1 - Generando texto con temperature 0.05
Seed: or prudentes eso es lo digo y 
Texto generado: or prudentes eso es lo digo y de la costa no se hesarandona la mscuadra ce la cabeza y como los mropositos de la costa no se desabandon de la costa de la contendiencia a lo carina y la costa de la contercia a los mngleses y los marineros y se representaba en la cabeza y como los mromeros se habian sido de la costa do se heceo es

------> Epoch: 1 - Generando texto con temperature 0.1
Seed: or prudentes eso es lo digo y 
Texto generado: or prudentes eso es lo digo y de la cosa de la costa ne la conta do se da de esta esta pompanada y desla casta no se depirado de la costa do se dece a ma marina y la costa de la costa ne la conta do se hecigiando la casla y la maro de la cobeza y como los parineros entaban en el rar a la cscuadra domo si el repto de la conta ne 

------> Epoch: 1 - Generando texto con temperature 0.2
Seed: or prudentes eso es lo digo y 
Texto generado: or prud

## Entregable

Completa los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2 usando El Quijote:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```

Asimismo, se proponen los siguientes aspectos opcionales para conseguir nota extra:

*   Experimentar con los textos de teatro en verso de Calderón de la Barca (¿es capaz el modelo de aprender las estructuras del teatro en verso?) o con alguno de los otros textos disponibles. También se puede probar con textos de vuestra elección.
*   Experimentar con distintos valores de SEQ_LENGTH.
*   Experimentar con los hiperparámetros del modelo o probar otro tipo de modelos como GRUs o *stacked* RNNs (RNNs apiladas).
*   Experimentar utilizando embeddings en vez de representaciones one-hot.
*   (Difícil) Entrenar un modelo secuencia a secuencia en vez de secuencia a carácter.


